In [1]:
pip install scx>=1.0.7


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'A1',
        'destination_name':'R1',
        'distance': 105,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R2',
        'distance': 256,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R3',
        'distance': 86,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R1',
        'distance': 240,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R2',
        'distance': 136,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R3',
        'distance': 198,
        'cost_per_mile':0.12,
    },
]

demand = [
    {
        'name':'R1', 
        'demand':2500
    },
    {
        'name':'R2', 
        'demand':4350
    },
    {
        'name':'R3', 
        'demand':3296
    },
]

assembly = [
    {
        'name':'A1',
        'limit':13000,
    },
    {
        'name':'A2',
        'limit':3500,
    },
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)
    # Calculate the variable cost of shipping for each item in tranport
    t['cost']=t['distance']*t['cost_per_mile']

In [5]:
# Initialize the my_model
my_model = Model(name="Blinky22", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['cost'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand']
    )

## Supply Constraint
for a in assembly:
    my_model.add_constraint(
        name=f"{a['name']}__assembly_supply",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==a['name']]) <= a['limit']
    ) 

# Solve the my_model
my_model.solve()

In [6]:
my_model.show_formulation()

Blinky22:
MINIMIZE
12.6*A1__R1__amt + 30.72*A1__R2__amt + 10.32*A1__R3__amt + 28.799999999999997*A2__R1__amt + 16.32*A2__R2__amt + 23.759999999999998*A2__R3__amt + 0.0
SUBJECT TO
R1__demand: A1__R1__amt + A2__R1__amt >= 2500

R2__demand: A1__R2__amt + A2__R2__amt >= 4350

R3__demand: A1__R3__amt + A2__R3__amt >= 3296

A1__assembly_supply: A1__R1__amt + A1__R2__amt + A1__R3__amt <= 13000

A2__assembly_supply: A2__R1__amt + A2__R2__amt + A2__R3__amt <= 3500

VARIABLES
A1__R1__amt Continuous
A1__R2__amt Continuous
A1__R3__amt Continuous
A2__R1__amt Continuous
A2__R2__amt Continuous
A2__R3__amt Continuous



In [7]:
# Show the outputs
my_model.show_outputs()

{'objective': 148746.72,
 'status': 'Optimal',
 'variables': {'A1__R1__amt': 2500.0,
               'A1__R2__amt': 850.0,
               'A1__R3__amt': 3296.0,
               'A2__R1__amt': 0.0,
               'A2__R2__amt': 3500.0,
               'A2__R3__amt': 0.0}}
